In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
sys.path.append('C:\\Users\\elisa\\Documents\\git\\mplstereonet')
sys.path.append('C:\\Users\\elisa\\Documents\\git\\gempy')
import mplstereonet
import gempy as gp
from importlib import reload
import os
sys.path.append('C:\\Users\\elisa\\Documents\\git\\remote-geomod')  # append local path to access rgeomod module
import pandas as pn
'''
import rgeomod

import gdal
import skimage
import scipy
import linecache
from mpl_toolkits.mplot3d import Axes3D
from copy import copy

import seaborn as sns
'''

ModuleNotFoundError: No module named 'mplstereonet'

In [ ]:
geo_data = gp.create_data([777000,794500,6345000,6349000,-1500,1000],[40,175,25],
                        path_i = "C:\\Users\\elisa\\Documents\\git\\gempy\\notebooks\\prototype_notebooks\\input_data\\Elisa_model-interfaces.csv",
                        path_o = "C:\\Users\\elisa\\Documents\\git\\gempy\\notebooks\\prototype_notebooks\\input_data\\Elisa_model-orientations.csv")

In [ ]:
other_interf = pn.read_csv('C:\\Users\\elisa\\Documents\\git\\MSc\\GeomodToGempy\\input_data\\All_Interfaces_Global_filtered_upscaling-socle.csv')

In [ ]:
geo_data.interfaces

In [ ]:
#sorting of lithologies
gp.set_series(geo_data,{"fault1":('E_Affenadou'), "fault2":('E_MasDieu'), "Trias_Series":('LIAS','TRIAS'), 
                        "Carbon_Series":('CARBO'), "Basement_Series":('basement')},
             order_formations=np.array(['E_Affenadou','E_MasDieu','LIAS','TRIAS','CARBO','basement']))

In [ ]:
gp.get_sequential_pile(geo_data)

In [ ]:
gp.plotting.plot_data(geo_data,direction='z')

In [ ]:
#Compiling of interpolator function
interp_data = gp.InterpolatorData(geo_data, compile_theano=True,
                                  dtype='float64',theano_optimizer='fast_run')

In [ ]:
lith_block, fault_block = gp.compute_model(interp_data)

In [ ]:
gp.plot_section(geo_data, lith_block[0], cell_number=25,  direction='y', plot_data=True)

## Topography

### 1. Import dtm (tif or ascii), compare extent and crop it according to extent of geo_data

In [ ]:
def import_dtm(path_dtm):
    '''returns: osgeo.gdal.Dataset'''
    #import_data_csv
    if path_dtm:
        dtm = gdal.Open(path_dtm)
    return dtm

def compare_extent(dtm, geo_data, show=True):
    dtm_extent, dtm_resolution = gdal2geodata_extent(dtm)
    cornerpoints_geo = get_cornerpoints(geo_data.extent)
    cornerpoints_dtm = get_cornerpoints(dtm_extent)
    
    if show:
        plt.style.use('bmh')
        plt.plot(cornerpoints_geo[:,0], cornerpoints_geo[:,1], 'ro', markersize = 12, label = 'Geo_data extent')
        plt.plot(cornerpoints_dtm[:,0], cornerpoints_dtm[:,1], 'gX',markersize = 11, label = 'DTM extent')
        plt.title('Extent comparison')
        plt.legend(loc=0, fancybox=True, shadow=True)
        plt.show()
        
    if np.any(cornerpoints_geo[:2]-cornerpoints_dtm[:2]) != 0:
        print('Extent of geo_data and DTM do not match. Use function cropDTMtogeodata to crop')
    else:
        print('Extent of geo_data and DTM match. You may continue!')
    
def gdal2geodata_extent(dtm):
    '''can return dtm.extent and dtm.resolution'''
    ulx, xres, xskew, uly, yskew, yres  = dtm.GetGeoTransform()
    z = dtm.ReadAsArray()
    if np.any(np.array([xskew,yskew]))!= 0:
        print('Obacht! DTM is not north-oriented. Stop.')
    lrx = ulx + (dtm.RasterXSize * xres)
    lry = uly + (dtm.RasterYSize * yres)
    res = np.array([(uly-lry)/(-yres),(lrx-ulx)/xres]).astype(int)   
    return np.array([ulx, lrx, lry, uly, z.min(), z.max()]).astype(int), res

def get_cornerpoints(extent):
    upleft = ([extent[0], extent[3]]) 
    lowleft = ([extent[0], extent[2]]) 
    upright = ([extent[1], extent[3]]) 
    lowright = ([extent[1], extent[2]]) 
    return np.array([upleft,lowleft,upright,lowright])

def cropDTM2geodata(path_dest, dtm, geo_data):
    new_bounds = (geo_data.extent[0], geo_data.extent[2], geo_data.extent[1], geo_data.extent[3])
    #destName = "C:\\Users\\elisa\\Documents\\git\\MSc\\GempyTopography\\cropped_DTM.tif"
    gdal.Warp(path_dest, dtm, options = gdal.WarpOptions(
        options = ['outputBounds'], outputBounds=new_bounds))
    return gdal.Open(path_dest)

def tif2xyz(path_dest, dtm):
    '''returns array with the x,y,z coordinates of the topography.'''
    shape = dtm.ReadAsArray().shape
    #print(shape)
    gdal.Translate(path_dest, dtm, options=gdal.TranslateOptions(options = ['format'],format = "XYZ"))
    xyz = pn.read_csv(path_dest, header=None, sep = ' ').as_matrix()
    return np.dstack([xyz[:,0].reshape(shape),xyz[:,1].reshape(shape),xyz[:,2].reshape(shape)])

##### 1.1 import dtm

In [ ]:
hallo = import_dtm(path_dtm='C:\\Users\\elisa\\Documents\\Ales\\DISTRIB_2018\\MNT\\MNT_Dx25m_ALES_L93.asc')

##### 1.2 compare extent

In [ ]:
compare_extent(hallo, geo_data)

##### 1.3 extent does not match --> crop

In [ ]:
path_dest="C:\\Users\\elisa\\Documents\\git\\MSc\\GempyTopography\\croppedhallo2.tif"
cropped = cropDTM2geodata(path_dest, hallo, geo_data)

##### 1.4 compare extent of new dtm with geo_data

In [ ]:
compare_extent(cropped, geo_data, show=True)

##### it does match now!

In [ ]:
# not sure if i need this but keep it here
path_dest = "C:\\Users\\elisa\\Documents\\git\\MSc\\GempyTopography\\XYZ.csv"
xyz = tif2xyz(path_dest, cropped)

## 2. Include topography in gempy plots

### 2.1 Plot as background in gp.plotting.plot_data (if direction = z)
- Problem: X and Y coordinates are not included (see array xyz)
- how combine it with seaborn plot of gempy (see plot_data in class plot_data_2D)

In [1]:
#Plot DEM
plt.figure(figsize=(12,4))
vdtm = plt.imshow(cropped.ReadAsArray())
plt.title("Topography")
cbar = plt.colorbar(vdtm)
cbar.set_label('elevation')

NameError: name 'plt' is not defined

### 2.2 Plot in plots of block model

In [ ]:
# lith block
lb = lith_block[0].reshape(geo_data.resolution)

In [ ]:
extent_dtm, res_dtm = gdal2geodata_extent(cropped)

The extent of the dtm and geo_data match now, but the resolution is still different:

In [ ]:
res_dtm, geo_data.resolution

In [ ]:
dtm = cropped.ReadAsArray()

In order to combine the datasets for plotting, there are two possible ways:
 - either the dtm has to be shrinked 
 - or the geo_data lith block has to be extented

But first of all, we need a function that creates an array with the same numbers for the same z-value related to the extent:

In [ ]:
def height_ind(dtm, zs):
    '''calculate 'indices': array with shape (ext1, ext2) where the values
    are similar if the elevation is similar.'''  
    #zs = np.linspace(geo_data.extent[4], geo_data.extent[5], lb.shape[2])
    #dz = geo_data.extent[5] - geo_data.extent[4])/geo_data.resolution[2]
    # dz is number of points in z direction
    dz = (zs[-1] - zs[0]) / len(zs)
    #2. substract minimum value (here -1000) from each value of dtm and divide it by dz
    dtm_v = (dtm - zs[0]) / dz
    return dtm_v.astype(int)


def calculate_geomap(lb, dtm, geo_data, plot=True):
    zs = np.linspace(geo_data.extent[4], geo_data.extent[5], lb.shape[2])
    indices = height_ind(dtm,zs)   
    geomap = np.zeros((lb.shape[0],lb.shape[1]))
    for x in range(lb.shape[0]):
        for y in range(lb.shape[1]):
            geomap[x,y] = lb[x,y,indices[x,y]]   
    if plot:
        plt.imshow(geomap, origin="lower", cmap=gp.plotting.colors.cmap, norm=gp.plotting.colors.norm)
        plt.title("Geological map")    
    return geomap  

def extend_lithblock(lb, factor):
    fertig2 = []
    for i in range(0,lb.shape[2]):
        lb_sub=lb[:,:,i]
        fertig = []
        for j in range(0, lb.shape[0]):
            y = np.repeat(lb_sub[j,:], factor)
            fertig = np.append(fertig, [y]*factor)
        fertig = fertig.reshape(lb.shape[0]*factor, lb.shape[1]*factor)
        fertig2.append(fertig)
        fertig2.append(fertig)
        fertig2.append(fertig)
        fertig2.append(fertig)
    return np.dstack(fertig2)

def mask_lith_block_above_topo(lb, geo_data, dtm):
    '''hier wird allen Werten die oberhalb der topographie 
    liegen ein minus eins zugewiesen und das dann maskiert'''
    zs = np.linspace(geo_data.extent[4], geo_data.extent[5], lb.shape[2])
    indices = height_ind(dtm,zs)
    
    geoblock = copy(lb)
    for x in range(lb.shape[0]):
        for y in range(lb.shape[1]):
            z = indices[x,y]
            geoblock[x,y,z:] = -1
    
    return np.ma.masked_where(geoblock < 0, geoblock)

In [ ]:
zs = np.linspace(geo_data.extent[4], geo_data.extent[5], geo_data.extent[2])
indices = height_ind(dtm_resized, zs)
plt.imshow(indices, origin="lower", cmap='viridis')
plt.title("Height indices", fontsize = 15)   

#### 2.2.1 resize DTM:

In [ ]:
#dtm
dtm_resized = skimage.transform.resize(dtm, (geo_data.resolution[0],geo_data.resolution[1]), preserve_range=True)
dtm_resized.shape

#### 2.2.2 Extent lith_block

In [ ]:
lb2 = extend_lithblock(lb, 4)
lb2.shape, dtm.shape

#### Plot

In [ ]:
geomap = calculate_geomap(lb,dtm_resized, geo_data, plot = False)
geomap2 = calculate_geomap(lb2, dtm, geo_data, plot = False)
plt.figure(1, figsize = (10,5))
plt.imshow(geomap, origin="lower", cmap=gp.plotting.colors.cmap, norm=gp.plotting.colors.norm)
plt.title("Geomap, geodata extent", fontsize = 15)   
plt.figure(2, figsize = (10,5))
plt.imshow(geomap2, origin="lower", cmap=gp.plotting.colors.cmap, norm=gp.plotting.colors.norm)
plt.title("Geomap, geotiff extent", fontsize = 15)   

In [ ]:
geoblock = mask_lith_block_above_topo(lb, geo_data, dtm_resized)
geoblock2 = mask_lith_block_above_topo(lb2, geo_data, dtm)

plt.figure(1, figsize = (10,5))
fig = plt.imshow(geoblock[25,:,:].T, origin="lower", cmap=gp.plotting.colors.cmap,norm=gp.plotting.colors.norm)
plt.title("Cross section, geodata extent", fontsize = 15)  
plt.figure(2, figsize = (10,5))
plt.imshow(geoblock2[100,:,:].T, origin="lower", cmap=gp.plotting.colors.cmap,norm=gp.plotting.colors.norm)
plt.title("Cross section, full extent", fontsize = 15)  

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(12,8))
ax = fig.gca(projection='3d')
surf = ax.plot_surface(xyz[:,:,0], xyz[:,:,1],xyz[:,:,2], cmap="viridis",
                       linewidth=0, antialiased=False)
#ax.view_init(azim=0, elev=90)
ax.set_zlim(-1000,1000)